In [1]:
import itertools
import numpy as np
from pyomo.environ import *
from pyomo.environ import value as pyoval
from pyomo.environ import Binary, NonNegativeReals
from typing import Sequence, Tuple
from collections import defaultdict

## VERTEX ENUMERATION

delta = min delta^k

delta^k = max delta

s.t.    A (d, z, theta^k) <= b
        theta = theta_nominal +/- theta_deviations
        z, theta >= 0

In [2]:
def vertex_enumeration(A:np.ndarray, b:np.ndarray, theta_nominal:list, theta_deviations:list, design_vectors:(np.ndarray, list[np.ndarray])=None, z_bounds:list[Tuple]=None):
    """
    A(d, z, theta) <= b
    Args:
        A: 2-D numpy array of coefficients of design, control, and uncertain parameters A(d, z, theta)
        b: 1-D numpy array of coefficients of constants
        theta_nominal: List of nominal values of uncertain parameters
        theta_deviations: List of bounds for uncertain parameters
        design_vectors: List of 1-D numpy arrays of design variable values (or a single array)
        z_bounds: List of bounds for control parameters

    Returns:

    """
    vertex_list = list(itertools.product(['+','-'], repeat=len(theta_nominal)))

    if not (isinstance(theta_nominal, list) and isinstance(theta_deviations, list) and all(isinstance(t, tuple) and len(t)==2 and all(isinstance(x, (int, float)) for x in t) for t in theta_deviations)):
        raise ValueError('Please provide a list (of length n_t) of tuples (of size 2) and containing only float or integer values!')
    elif len(theta_nominal) != len(theta_deviations):
        raise ValueError('Inconsistent input for nominal theta values and theta bounds')

    n_t = len(theta_nominal)
    n_d = design_vectors[0].shape[0] if isinstance(design_vectors, list) else design_vectors.shape[0] if isinstance(design_vectors, np.ndarray) else 0
    n_c = A.shape[1] - n_t - n_d

    if n_c < 0:
        raise ValueError('Inconsistent coefficients input')       
    if isinstance(z_bounds, list):        
        if not (len(z_bounds) == n_c and all(isinstance(t, tuple) and len(t)==2 and all(isinstance(x, (int, float)) for x in t) for t in z_bounds)):
            raise ValueError('Please provide a list (of length n_c) of tuples (of size 2) and containing only float or integer values!')

    def evaluate_delta_fixed_design(z_b:list[Tuple]=None, design_vector:np.ndarray=None):
        model_min = None
        delta_min = None
        active_set_list = None
        vertex_min = None

        for vertex in vertex_list:
            m = ConcreteModel()
            m.Ai = RangeSet(A.shape[0])
            m.Aj = RangeSet(A.shape[1])
            m.i = RangeSet(n_d + n_c + n_t)

            m.var = Var(m.i, within=NonNegativeReals) # structure (d,z,theta)
            m.delta = Var(within=NonNegativeReals)

            # system constraints: A(d,z,theta) <= b
            def Ax_leq_b_rule(instance, i):
                return sum(A[i-1, j-1]*instance.var[j] for j in instance.Aj) <= b[i-1]
            m.Ax_leq_b = Constraint(m.Ai, rule=Ax_leq_b_rule)

            # Add constraint for fixing first vars to the design value: d = dE
            if isinstance(design_vector, np.ndarray):
                m.d = RangeSet(n_d)
                def fix_design_rule(instance, d):
                    return instance.var[d] == design_vector[d-1]
                m.d_eq = Constraint(m.d, rule=fix_design_rule)

            if isinstance(z_b, list):
                m.z = RangeSet(n_c)
                def z_LB_rule(instance, z):
                    return z_bounds[z-1][0] <= instance.var[n_d+z]
                m.z_lb = Constraint(m.z, rule=z_LB_rule)

                def z_UB_rule(instance, z):
                    return instance.var[n_d+z] <= z_bounds[z-1][1]
                m.z_ub = Constraint(m.z, rule=z_UB_rule)

            # theta constraints: theta = theta_nominal +/- theta_deviations * delta
            m.constraints = ConstraintList()
            theta_index = n_d + n_c + 1
            for i in range(len(theta_nominal)):
                if vertex[i]=='+':
                    expr = m.var[theta_index] == theta_nominal[i] + theta_deviations[i][1]*m.delta
                else:
                    expr = m.var[theta_index] == theta_nominal[i] - theta_deviations[i][0]*m.delta
                m.constraints.add(expr)
                theta_index+=1

            # Objective
            m.objective = Objective(expr=m.delta, sense=maximize)
            solver = SolverFactory('gurobi')
            solver.solve(m)

            if delta_min is None or pyoval(m.delta) < delta_min:
                delta_min = pyoval(m.delta)
                active_set_list = [f'f{c}' for c in m.Ax_leq_b if round(pyoval(m.Ax_leq_b[c].body),5) == round(pyoval(m.Ax_leq_b[c].upper),5)]
                model_min = m
                vertex_min = vertex

        return model_min, active_set_list, vertex_min, delta_min

    if isinstance(design_vectors, list):
        model_min, active_set_min, vertex_min, delta_min = [defaultdict(dict) for _ in range(len(design_vectors))]
        for i in range(len(design_vectors)):
            model_min[i], active_set_min[i], vertex_min[i], delta_min[i] = evaluate_delta_fixed_design(z_b=z_bounds, design_vector=design_vectors[i])
        return model_min, active_set_min, vertex_min, delta_min
    elif isinstance(design_vectors, np.ndarray):
        return evaluate_delta_fixed_design(z_b=z_bounds, design_vector=design_vectors)
    else:
        return evaluate_delta_fixed_design(z_b=z_bounds)

In [3]:
theta_nominal = [620, 388, 583, 313]
theta_deviations = [(10, 10), (10, 10), (10, 10), (10, 10)]

A = np.array([
    [-2/3, 0, 1, 0, 0],
    [-0.5, -0.75, -1, -1, 0],
    [1, -1.5, -2, -1, -2],
    [1, -1.5, -2, -1, 0],
    [-1, 1.5, 2, 1, 3]
])

b = np.array([350, -1388.5, -2830, -2044, 3153])

d0 = np.array([0.1])

model_VE, active_set_list, vertex, delta = vertex_enumeration(A=A, b=b, theta_nominal=theta_nominal, theta_deviations=theta_deviations)

print(f'Flexibility index calculated through vertex enumeration: {delta}')
print(f'Active set for vertex {vertex}: {active_set_list}')

Flexibility index calculated through vertex enumeration: 0.5600000000000023
Active set for vertex ('-', '-', '-', '-'): ['f1', 'f3']


## ACTIVE SET STRATEGY

In [4]:
def active_set_method(A:np.ndarray, b:np.ndarray, theta_nominal:list, theta_deviations:list, design_vectors:np.ndarray | list[np.ndarray]=None, z_bounds:list[Tuple]=None, bigM:float=1e3):
    """
    From:
    A(d, z, theta) <= b
    z_lb <= z <= z_ub
    theta_nominal - delta * theta_deviations_lb <= theta <= theta_nominal + delta * theta_deviations_ub
    
    Convert to:
    h(d, z, theta, slack, dual, binary) == b_h
    g(z, theta, slack, dual, binary, delta) <= b_g

    Args:
        A: Coefficient matrix of system equations
        b: Vector of constant values
        theta_nominal: Nominal values of uncertain parameters
        theta_deviations: Allowable changes from the nominal value (lower bound, upper bound)
        design_vectors: Values of design variables
        z_bounds: Bounds on control variables
        bigM: 

    Returns:
        m: Pyomo model
        delta: Flexibility index value
    """
    if not (isinstance(theta_nominal, list) and isinstance(theta_deviations, list) and all(isinstance(t, tuple) and len(t)==2 and all(isinstance(x, (int, float)) for x in t) for t in theta_deviations)):
        raise ValueError('Please provide a list (of length n_t) of tuples (of size 2) and containing only float or integer values!')
    elif len(theta_nominal) != len(theta_deviations):
        raise ValueError('Inconsistent input for nominal theta values and theta bounds')
    
    n_t = len(theta_nominal)
    n_d = design_vectors[0].shape[0] if isinstance(design_vectors, list) else design_vectors.shape[0] if isinstance(design_vectors, np.ndarray) else 0
    n_c = A.shape[1] - n_t - n_d
    if n_c < 0:
        raise ValueError('Inconsistent coefficients input')
    # print(f'num_theta={n_t}, num_design={n_d}, num_control={n_c}')
    # print(f'num_constraints={A.shape[0]}')
    if isinstance(z_bounds, list):        
        if not (len(z_bounds) == n_c and all(isinstance(t, tuple) and len(t)==2 and all(isinstance(x, (int, float)) for x in t) for t in z_bounds)):
            raise ValueError('Please provide a list (of length n_c) of tuples (of size 2) and containing only float or integer values!')
    
    # Reformulate A(d, z, theta) <= b to h(d, z, theta, slack, dual, binary) == b_h
    # h1(d, z, theta, slack) == b               for all system constraints
    # h2(dual) == 0 (from dA/dz = 0)            for all control variables
    # h3(dual, binary) == [1, n_c+1]'           2 constraints
    # d == dE                                   for all design variables - if provided
    # b_h = [b, 0*n_c, 1, n_c+1, dE (if provided)]'
    h1 = np.concatenate([A, np.diag([1]*A.shape[0]), np.zeros((A.shape[0], 2*A.shape[0]))], axis=1)
    h2 = np.concatenate([np.zeros((n_c, A.shape[1]+A.shape[0])), A[:, n_d:n_d+n_c].T, np.zeros((n_c, A.shape[0]))], axis=1)
    h3 = np.concatenate([np.zeros((2, A.shape[1]+A.shape[0])), np.stack([np.ones(A.shape[0]).T, np.zeros(A.shape[0]).T], axis=0), np.stack([np.zeros(A.shape[0]).T, np.ones(A.shape[0]).T], axis=0)], axis=1)
    h = np.concatenate([h1, h2, h3], axis=0)
    b_h = np.concatenate([b, np.zeros(n_c), np.array([1]), np.array([n_c+1])], axis=0)
    
    # Add inequalities for the active set reformulation g(z, theta, slack, dual, binary, delta) <= b_g
    # z <= z_UB                                             Upper bounds for all control variables
    # -z <= -z_LB                                           Lower bounds for all control variables
    # theta - theta_bound*delta <= theta_nominal            Upper bounds for all uncertain parameters
    # -theta - theta_bound*delta <= -theta_nominal          Lower bounds for all uncertain parameters
    # slack + bigM*binary <= bigM                           # bigM reformulation of complementary conditions
    # dual - binary <= 0                                    bigM reformulation of complementary conditions
    g1 = np.concatenate([np.concatenate([np.diag([1]*n_t), np.diag([-1]*n_t)], axis=0), np.zeros((2*n_t, 3*A.shape[0])), np.stack([-np.array([i[1] for i in theta_deviations]), -np.array([i[1] for i in theta_deviations])], axis=0).reshape(-1,1)], axis=1)
    g2 = np.concatenate([np.zeros((A.shape[0], n_t)), np.diag([1]*A.shape[0]), np.diag([0]*A.shape[0]), np.diag([bigM]*A.shape[0]), np.zeros(A.shape[0]).reshape(-1,1)], axis=1)
    g3 = np.concatenate([np.zeros((A.shape[0], n_t)), np.diag([0]*A.shape[0]), np.diag([1]*A.shape[0]), np.diag([-1]*A.shape[0]), np.zeros(A.shape[0]).reshape(-1,1)], axis=1)
    g = np.concatenate([g1, g2, g3], axis=0)
    b_g = np.concatenate([np.array(theta_nominal), -np.array(theta_nominal), np.array([bigM]*A.shape[0]), np.array([0]*A.shape[0])], axis=0)
    g_shift=n_d+n_c
    
    def evaluate_delta_fixed_design(h, b_h, g, b_g, g_shift, z_b:np.ndarray=None, design_vector:np.ndarray=None):
        if isinstance(design_vector, np.ndarray):
            h4 = np.concatenate([np.diag([1]*n_d), np.zeros((n_d, n_c+n_t+3*A.shape[0]))], axis=1)
            h_new, b_h_new = np.concatenate([h, h4], axis=0),np.concatenate([b_h, design_vector], axis=0)
            # b_h_new = np.concatenate([b_h, design_vector], axis=0)
        else:
            h_new, b_h_new= h, b_h
            # b_h_new = b_h
        if isinstance(z_b, list):
            g0 = np.concatenate([np.concatenate([np.diag([1]*n_c), np.diag([-1]*n_c)], axis=0), np.zeros((2*n_c, n_t+3*A.shape[0]+1))], axis=1)
            gz = np.zeros((2*(n_t+A.shape[0]), n_c))
            b0 = np.array([i[1] for i in z_bounds]+[-i[0] for i in z_bounds])
            g_new, b_g_new = np.concatenate([g0, np.concatenate([gz, g], axis=1)], axis=0), np.concatenate([b0, b_g], axis=0)
            g_shift=n_d
        else:
            g_new, b_g_new = g, b_g
        # print(f'Shape of h matrix {h.shape}')
        # print(f'Shape of b_h matrix {b_h.shape}')
        # print(f'Shape of g matrix {g.shape}')
        # print(f'Shape of b_g matrix {b_g.shape}')
                
        binary_indexes = list(range(A.shape[1]+2*A.shape[0]+1, A.shape[1]+3*A.shape[0]+1))
        def var_domain_rule(instance, i):
            return Binary if i in binary_indexes else NonNegativeReals
        
        m = ConcreteModel()
        m.v = RangeSet(n_d+n_c+n_t+3*A.shape[0]+1)
        m.var = Var(m.v, domain=var_domain_rule)        # order of variables (design, control, theta, slack, dual, binary, delta)
        
        m.hi = RangeSet(h_new.shape[0])
        m.hj = RangeSet(h_new.shape[1])
        m.gi = RangeSet(g_new.shape[0])
        m.gj = RangeSet(g_new.shape[1])

        def h_eq_bh_rule(instance, i):
            return sum(h_new[i-1, j-1]*instance.var[j] for j in instance.hj) == b_h_new[i-1]
        m.h_eq_bh = Constraint(m.hi, rule=h_eq_bh_rule)
        
        def g_leq_bg_rule(instance, i):
            return sum(g_new[i-1, j-1]*instance.var[j+g_shift] for j in instance.gj) <= b_g_new[i-1]
        m.g_leq_bg = Constraint(m.gi, rule=g_leq_bg_rule)
        
        # Objective
        m.objective = Objective(expr=m.var[len(m.v)], sense=minimize)
        solver = SolverFactory('gurobi')
        solver.solve(m)
        
        actives = [j-(A.shape[1]+2*A.shape[0]) for j in binary_indexes if pyoval(m.var[j]) == 1] # Adjust for constraint number
        return m, pyoval(m.var[len(m.v)]), actives
    
    if isinstance(design_vectors, list):
        m, delta, active_set = [defaultdict(dict) for _ in range(3)]
        for i in range(len(design_vectors)):
            m[i], delta[i], active_set[i] = evaluate_delta_fixed_design(h=h, b_h=b_h, g=g, b_g=b_g, g_shift=g_shift, z_b=z_bounds, design_vector=design_vectors[i])
    elif isinstance(design_vectors, np.ndarray):
        m, delta, active_set = evaluate_delta_fixed_design(h=h, b_h=b_h, g=g, b_g=b_g, g_shift=g_shift, z_b=z_bounds, design_vector=design_vectors)
    else:
        m, delta, active_set = evaluate_delta_fixed_design(h=h, b_h=b_h, g=g, b_g=b_g, g_shift=g_shift)
    
    return m, delta, active_set

In [5]:
# EXAMPLE 1 Floudas and Grossmann (1987): https://www.sciencedirect.com/science/article/pii/0098135487870114
theta_nominal = [620, 388, 583, 313]
theta_deviations = [(10, 10), (10, 10), (10, 10), (10, 10)]

A = np.array([
    [-2/3, 0, 1, 0, 0],
    [-0.5, -0.75, -1, -1, 0],
    [1, -1.5, -2, -1, -2],
    [1, -1.5, -2, -1, 0],
    [-1, 1.5, 2, 1, 3]
])

b = np.array([350, -1388.5, -2830, -2044, 3153])

In [6]:
model, flex_index, act_set = active_set_method(A=A, b=b, theta_nominal=theta_nominal, theta_deviations=theta_deviations)

In [7]:
flex_index

0.5600000000000023

In [8]:
act_set

[1, 3]

In [9]:
# # # EXAMPLE 1 Pistikopoulos and Grossmann (1988): https://www.sciencedirect.com/science/article/pii/0098135488800103
# theta_nominal = [2]
# theta_deviations = [(2,2)]
# 
# A = np.array([
#     [1, -3, 1, -1],
#     [0, 1, -1, -1/3],
#     [-1, 0, 1 ,1]
# ])
# 
# b = np.array([0, -1/3, 1])
# 
# d = np.array([2.2, 0.7333])

In [10]:
# model, flex_index, act_set = active_set_method(A=A, b=b, theta_nominal=theta_nominal, theta_deviations=theta_deviations, design_vectors=d)

In [11]:
# flex_index